In [1]:
import os
import unicodedata
import pandas as pd
import calendar
import datetime
import re
import requests
from bs4 import BeautifulSoup as bs

# import WikiNewsNetwork as wnn

In [2]:
def parse(request):
    """
    Query Wikipedia API with specified parameters to parse data.
    Parameters
    ----------
    request : dict
        API call parameters.
    Raises
    ------
    ValueError
        Raises error if returned by API.
    Yields
    ------
    dict
        Subsequent dicts of json API response.
    """
    request['action'] = 'parse'
    request['format'] = 'json'
    lastContinue = {}
    while True:
        # Clone original request
        req = request.copy()
        # Modify with values from the 'continue' section of the last result.
        req.update(lastContinue)
        # Call API
        result = requests.get(
            'https://de.wikipedia.org/w/api.php', params=req).json()
        if 'error' in result:
            print('ERROR')
            # print(result['error'])
            raise ValueError(result['error'])
        if 'warnings' in result:
            print(result['warnings'])
        if 'parse' in result:
            yield result['parse']
        if 'continue' not in result:
            break
        lastContinue = result['continue']


In [3]:
from bs4 import BeautifulSoup

# assume `soup` is the BeautifulSoup object for your page
h3_ul_pairs = []

for m in [  'Januar 2022', 'Februar 2022', 'März_2022', 'April_2022', 'Mai_2022', 'Juni_2022', 
          'Juli_2022', 'August_2022', 'September_2022', 'Oktober_2022', 'November_2022', 'Dezember_2022']:
  print(m)
  params = {'page':  m, 'prop': 'text'}
  # Get data from current events page for that month
  scr = list(parse(params))
  doc = bs(scr[0]['text']['*'], 'html.parser')

  # find the first h3 element
  h3 = doc.find('h3')
  # temp = h3.find_next('h3')

  while h3:
      # find the next h3 element
      next_h3 = h3.find_next('h3')

      # find all ul elements between this h3 and the next h3
      uls = []
      print(len(h3.find_next_siblings()))
      for el in h3.find_next_siblings():
          print(el.name)
          if el.name == 'ul':
              for link in [i.find('a') ['href'] for i in el.findAll('li')]:
                h3_ul_pairs.append((h3.find('span', {"class": "mw-headline"}).get('id'), link))

              break
          if el.name == 'h3':
              break


      # move on to the next h3 element
      h3 = next_h3
    

Januar 2022
72
div
ul
68
ul
66
ul
64
ul
62
div
ul
59
ul
57
ul
55
ul
53
ul
51
ul
49
ul
47
ul
45
ul
43
ul
41
div
ul
38
ul
36
ul
34
div
ul
31
ul
29
ul
27
ul
25
ul
23
ul
21
ul
19
h3
18
h3
17
div
ul
14
ul
12
ul
10
ul
8
ul
Februar 2022
62
ul
60
ul
58
ul
56
div
ul
53
ul
51
ul
49
ul
47
ul
45
ul
43
ul
41
h3
40
ul
38
ul
36
ul
34
ul
32
ul
30
ul
28
ul
26
ul
24
ul
22
ul
20
h3
19
ul
17
div
ul
14
ul
12
ul
10
ul
8
ul
März_2022
73
ul
71
ul
69
ul
67
ul
65
ul
63
ul
61
ul
59
ul
57
div
div
ul
53
ul
51
div
ul
48
ul
46
div
ul
43
ul
41
ul
39
div
ul
36
div
ul
33
ul
31
div
ul
28
ul
26
ul
24
ul
22
h3
21
ul
19
ul
17
ul
15
ul
13
h3
12
ul
10
ul
8
ul
April_2022
59
ul
57
ul
55
ul
53
ul
51
ul
49
ul
47
ul
45
ul
43
ul
41
ul
39
ul
37
h3
36
h3
35
h3
34
h3
33
ul
31
h3
30
h3
29
ul
27
ul
25
h3
24
ul
22
ul
20
ul
18
ul
16
ul
14
ul
12
ul
10
ul
8
ul
Mai_2022
79
h3
78
ul
76
ul
74
ul
72
ul
70
ul
68
ul
66
ul
64
ul
62
div
div
ul
58
ul
56
ul
54
div
ul
51
ul
49
div
ul
46
ul
44
ul
42
ul
40
div
ul
37
ul
35
ul
33
ul
31
div
ul
28
div
ul
2

In [ ]:
len(h3_ul_pairs)

699

In [4]:
df = pd.DataFrame(h3_ul_pairs, columns=['date', 'link'])
df.head()

,date,link
0,"Samstag,_1._Januar_2022",/wiki/Berlin
1,"Samstag,_1._Januar_2022",/wiki/Berlin
2,"Samstag,_1._Januar_2022",/wiki/Bern
3,"Samstag,_1._Januar_2022",/wiki/Br%C3%BCssel
4,"Samstag,_1._Januar_2022",/wiki/Frankfurt_am_Main


In [5]:
def transform_date(date):
  components = date.split('_')
  month = {"Januar": "01", "Februar": "02", "März": "03", 
           "April": "04", "Mai": "05", "Juni": "06",
           "Juli": "07", "August": "08", "September": "09", 
           "Oktober": "10", "November": "11", "Dezember": "12"}
  components[1] = components[1].split('.')[0]
  if len(components[1])==1:
    components[1] = '0' + components[1]
  return '2022' + month[components[2]]+ components[1]
df['date'] = df['date'].apply(lambda x: transform_date(x))

In [6]:
df['page_title'] = df['link'].apply(lambda x: x[6:])
df = df.drop('link', axis=1)
df

,date,page_title
0,20220101,Berlin
1,20220101,Berlin
2,20220101,Bern
3,20220101,Br%C3%BCssel
4,20220101,Frankfurt_am_Main
...,...,...
695,20221229,Jerusalem
696,20221230,Berlin
697,20221230,Peking
698,20221231,Bogot%C3%A1


In [7]:

from google.colab import files
df.to_csv('events_ge.csv') 
files.download('events_ge.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>